# Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


# Data Analysis and Modeling
OSEMN Framework

* Obtain
* Scrub
* Explore
* Model
* INterpret

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mlp
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import Image
import os
import sklearn
pd.set_option('display.max_columns', 50)

In [ ]:
# Modeling & Evaluation Packages
from statsmodels.formula.api import ols
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import Imputer, PolynomialFeatures, StandardScaler
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.model_selection import StratifiedKFold, KFold, cross_val_score, GridSearchCV, learning_curve, train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import gc
import re

# Obtain

## Data Frame

In [ ]:
df = pd.read_csv('dataset.csv')

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# for col in df.columns: # preliminary view of value counts per column
#     try:
#         print(col, df[col].value_counts()[:10])
#     except:
#         print(col, df[col].value_counts())
#         # If there aren't 5+ unique values for a column the first print statement
#         # will throw an error for an invalid idx slice
#     print('\n') # Break up the output between columns

# Scrub

## Multicollinearity

In [ ]:
corr = df.corr() # analyzing correlation
# corr
fig, ax = plt.subplots(figsize=(18,26))
mask = np.triu(np.ones_like(corr, dtype=np.bool))
sns.heatmap(corr, mask=mask, square=True, annot=True, cmap="YlGnBu")
#xticklabels=labels, yticklabels=labels)
#plt.xticks(rotation=-45, fontsize=16)
ax.patch.set_edgecolor('black')  
ax.patch.set_linewidth('1')
ax.set_title("Correlation & Heat Map", fontsize=15, fontfamily="serif")
plt.show()

## Casting Datatypes 

In [ ]:
numfeats = df.select_dtypes('number').columns # selecting the numerical columns for observation
numfeats

In [ ]:
# model_cols = ['account length', 'area code', 'number vmail messages',
#        'total day minutes', 'total day calls', 'total day charge',
#        'total eve minutes', 'total eve calls', 'total eve charge',
#        'total night minutes', 'total night calls', 'total night charge',
#        'total intl minutes', 'total intl calls', 'total intl charge',
#        'customer service calls'] # Checking distributions

# for column in model_cols:
#     df[column].plot.hist(density=True) # Creating a Histogram
#     df[column].plot.kde(label=column) # Creating a KDE plot
#     plt.legend() # Create Legends
#     plt.show()

In [ ]:
# ID column is phone number
catfeats = ['state', 'area code', 'international plan', 'voice mail plan', 'churn'] # recast categorical
for col in catfeats:
    df[col] = df[col].astype('category')

In [ ]:
df.info()

## Outliers

In [ ]:
df.describe() # addressing outliers on account length

In [ ]:
# IQR Outlier Removal Function
def iqr_outlier_rem(df, col_name):
    start_len = len(df)
    Q1 = df[col_name].quantile(0.25)
    Q3 = df[col_name].quantile(0.75)
    IQR = Q3-Q1 # Finding interquartile range
    lower_threshold  = Q1-1.5*IQR
    upper_threshold = Q3+1.5*IQR
    new_df = df.loc[(df[col_name] > lower_threshold) & (df[col_name] < upper_threshold)]
    print(f'There were {start_len - len(new_df)} outliers removed from {col_name}')
    return new_df

In [ ]:
df = iqr_outlier_rem(df, 'account length')

# Explore - EDA

In [ ]:
# plt.figure(figsize=(16,8))
# plt.subplot(221)
# sns.scatterplot(df['total day minutes'], df['total day charge'])
# plt.title('Total Day Minutes X Day Charge')

# plt.subplot(222)
# sns.scatterplot(df['total eve minutes'], df['total eve charge'])
# plt.title('Total eve Minutes X Eve Charge')

# plt.subplot(223)
# sns.scatterplot(df['total night minutes'], df['total night charge'])
# plt.title('Total Night Minutes X Night Charge')

# plt.subplot(224)
# sns.scatterplot(df['total intl minutes'], df['total intl charge'])
# plt.title('Total Intl Minutes X Intl Charge')

# # #plt.title('Sqft_living15 and Price')

# plt.subplots_adjust(hspace=0.35)
# plt.show()

In [ ]:
df.head()

In [ ]:
ax=sns.countplot(x="churn",data=df)
for p in ax.patches:
        ax.annotate('{:.2f}%'.format((p.get_height()/df.shape[0])*100), (p.get_x()+0.3, p.get_height()))

In [ ]:
ac = df.groupby(["area code", "churn"]).size().unstack().plot(kind='bar', stacked=False,figsize=(6,5))
for i in ac.patches:
    #ac.annotate('{:.2f}%'.format((p.get_height()/df.shape[0])*100), (p.get_x()+0.3, p.get_height()))
    ac.text(i.get_x()+0.05, i.get_height()+20,str(i.get_height()))

In [ ]:
df.groupby(["state", "churn"]).size().unstack().plot(kind='bar', stacked=True, figsize=(30,10)); # churn by statea

In [ ]:
df.groupby(["area code", "churn"]).size().unstack().plot(kind='bar', stacked=True, figsize=(5,5)); # churn by area code

In [ ]:
cs = df.groupby(["customer service calls", "churn"]).size().unstack().plot(kind='bar', stacked=False,figsize=(12,6))
for i in cs.patches:
    cs.text(i.get_x()+0.05, i.get_height()+20,int(i.get_height()))

In [ ]:
ip=df.groupby(["international plan", "churn"]).size().unstack().plot(kind='bar', stacked=False,figsize=(6,5))
for i in ip.patches:
    ip.text(i.get_x()+0.05, i.get_height()+20,str(i.get_height()))

In [ ]:
# df['phone_number'].value_counts() # no duplicates

In [ ]:
df.info()

In [ ]:
# dup = df[df.id.duplicated(keep=False)]
# print('Renovated vs. Non-renovated homes Resold (2014-2015)')
# print(dup['yr_renovated'].value_counts())
# dup['yr_renovated'].hist(figsize = (10,10), color = "green"); #let's make a histogram to compare
# plt.title('Renovated vs. Non-renovated homes Resold (2014-2015)', fontdict={'fontsize': 16});

## Dropping Features

In [ ]:
df_fresh = df.copy()

In [ ]:
# dup = df[[df['phone_number'].duplicated(keep=False)]]
# dup.head()
# #dup['phone_number'].value_counts()

In [ ]:
df = df.drop(['phone number', 'total day minutes', 'total eve minutes', 'total night minutes', 'total intl minutes'], axis=1)

In [ ]:
df.info()

In [ ]:
# One Hot Encode
df = pd.get_dummies(df, drop_first=True)

In [ ]:
# cleaning column names
subs = [(' ', '_'),('.0',''),('.','')]

def col_formatting(col):
    for old, new in subs:
        col = col.replace(old,new)
    return col

df.columns = [col_formatting(col) for col in df.columns]

In [ ]:
df.info()

In [ ]:
# Defining Variables
outcome = 'churn_True'
x_cols = list(df.columns)
x_cols.remove(outcome)
# Test Train Split
train, test = train_test_split(df)
print(len(train), len(test))

In [ ]:
# Fitting the model
predictors = '+'.join(x_cols)
formula = outcome + '~' + predictors
model = ols(formula=formula, data=train).fit()
model.summary()

In [ ]:
# Function to create two plots - normality / homoscedasticity
def model_visual(residuals):
# QQ Plot
    '''Checking residuals for normality'''
    fig, axes = plt.subplots(ncols=2, figsize=(20, 5))
    sm.graphics.qqplot(model.resid, fit=True, line='45', ax=axes[0])
    axes[0].set_title('QQ Plot Normality Check', fontsize=20)
    axes[0].set_xlabel('Theoretical Quantiles', fontsize=15)
    axes[0].set_ylabel('Sample Quantiles', fontsize=14)
# Homoscedasticity
    '''Checking Homoscedasticity assumption'''
    ax=axes[1]
    ax.scatter(train['churn_True'], model.resid)
    ax.axhline(0, color='red')
    axes[1].set_title('Homoscedasticity Check', fontsize=20)
    axes[1].set_xlabel('Price', fontsize=15)
    axes[1].set_ylabel('Model Residuals', fontsize=14)
    plt.show();
    return model_visual

In [ ]:
# model_visual(model.resid)

In [ ]:
# VIF Scores
x = df.drop('churn_True', axis = 1)
x = sm.add_constant(x)
vif = [variance_inflation_factor(x.values,i) for i in range(x.shape[1])]
vif_results = pd.Series(dict(zip(x.columns, vif)))

# Identifying VIF
threshold = 6
rem_vif = list(vif_results[vif_results > threshold].index)
if 'const' in rem_vif:
    rem_vif.remove('const')
rem_vif

In [ ]:
df = df.drop(['number_vmail_messages', 'voice_mail_plan_yes'], axis=1)

In [ ]:
# # Defining Variables
# outcome = 'churn_True'
# x_cols = list(df.columns)
# x_cols.remove(outcome)
# # Test Train Split
# train, test = train_test_split(df)
# print(len(train), len(test))

# # Fitting the model
# predictors = '+'.join(x_cols)
# formula = outcome + '~' + predictors
# model = ols(formula=formula, data=train).fit()
# model.summary()